# 📘 TCS Stock Predictive Analysis in Jupyter Notebook

## 📦 Step 1: Import Required Libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')


## 📥 Step 2: Load and Prepare Data

In [ ]:

stock_history = pd.read_csv('TCS_stock_history.csv')
stock_history['Date'] = pd.to_datetime(stock_history['Date'])
stock_history.sort_values('Date', inplace=True)
stock_history['Close_Lag1'] = stock_history['Close'].shift(1)
stock_history['Close_Lag5'] = stock_history['Close'].shift(5)
stock_history['MA20'] = stock_history['Close'].rolling(window=20).mean()
stock_history['MA50'] = stock_history['Close'].rolling(window=50).mean()
stock_history['Volume'] = stock_history['Volume']
stock_history.dropna(inplace=True)


## 📊 Evaluation Function

In [ ]:

def evaluate_model(y_true, y_pred, model_name):
    print(f"\n📊 {model_name} Performance:")
    print(f"MAE: {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
    print(f"R²: {r2_score(y_true, y_pred):.4f}")


## 🔮 1. Next-Day Close Price Prediction (Linear Regression)

In [ ]:

features = ['Close_Lag1', 'Close_Lag5', 'MA20', 'MA50']
X = stock_history[features]
y = stock_history['Close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

evaluate_model(y_test, y_pred, "Linear Regression")


## 📈 2. Trend Classification: Up/Down Movement (Random Forest)

In [ ]:

stock_history['Target'] = (stock_history['Close'].shift(-1) > stock_history['Close']).astype(int)
X = stock_history[features]
y = stock_history['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d')
plt.title("Confusion Matrix")
plt.show()


## 📉 3. Volatility Forecasting (Classification)

In [ ]:

stock_history['Volatility'] = stock_history['Close'].rolling(window=10).std()
stock_history['High_Volatility'] = (stock_history['Volatility'] > stock_history['Volatility'].quantile(0.75)).astype(int)
X = stock_history[features]
y = stock_history['High_Volatility']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
print("Volatility Accuracy:", accuracy_score(y_test, rf.predict(X_test)))


## 📊 4. 3-Day Ahead Close Price Forecast

In [ ]:

stock_history['Close_3D_Future'] = stock_history['Close'].shift(-3)
stock_history.dropna(inplace=True)

X = stock_history[features]
y = stock_history['Close_3D_Future']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

evaluate_model(y_test, y_pred, "3-Day Forecast")


## 🧠 5. Time Series Forecasting with ARIMA

In [ ]:

ts = stock_history.set_index('Date')['Close']
model = ARIMA(ts, order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=10)

ts.plot(label='Actual', figsize=(12,6))
forecast.plot(label='Forecast', color='red')
plt.legend()
plt.title("ARIMA Forecast - Next 10 Days")
plt.show()
